<a href="https://colab.research.google.com/github/kalyan8324/Deep_learning/blob/main/bank_Churn_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'bank-customer-churn-modeling:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F17473%2F22922%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240612%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240612T081949Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D52690e9a39524be09bf238c75cbce3cc27b0a78151426df95e0ce05966d501b2d8ca719bdc6dc527dc6ebe106532dd363b29e6467af897f95570bf635480e75e1630327441dc3aa10f35acabadc9457be5146da761471dc958614a1e2ccfcbe186bce89e5bdb02721da86adda9099e3fdaba47993f3ab42fda981843d4a52206afbdf16140d4e54af3934445e03f5faa1752bb0f906fa436bde71f99e56026bd622a00c81d424e6f7e5546ba56c78f617927a39f1eea0fbafcdf3d04358e1299642bef63e06e45e7ef7b3306e92e60ffacb7a347be15efd660427dbff877c5b81a1d3e5cced14ba213b52296c6b0f662192dd7daaef0bb295bcdb72e7408e4a5'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Bank Customer Churn_Prediction

In [ ]:
# import required librarys
import numpy as np
import pandas as pd
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt

<!-- import data set from kaggle  -->

In [ ]:
# Dataset path: "/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv"
bank = pd.read_csv("/kaggle/input/bank-customer-churn-modeling/Churn_Modelling.csv", index_col="RowNumber")
bank.head()

1. we are deleteing un nessasary columns in data sets

In [ ]:
bank.drop(['CustomerId', 'Surname'], axis = 1, inplace = True)
bank.head()

checking datatypes

In [ ]:
bank.dtypes

In [ ]:
bank['Gender'].unique()

In [ ]:
import matplotlib.pyplot as plt
tenure_churn_yes = bank[bank.Exited == 0].Tenure
tenure_churn_no = bank[bank.Exited == 1].Tenure

plt.xlabel('Tenure')
plt.ylabel('no of customer')
plt.title("Customer Exits or ")
plt.hist([tenure_churn_yes, tenure_churn_no], color = ['green', 'red'], label = ['churn = yes', 'churn = no'])
plt.legend()

In [ ]:
Salary_churn_yes = bank[bank.Exited ==0].EstimatedSalary
Salary_churn_no = bank[bank.Exited == 1].EstimatedSalary
plt.xlabel('Tenure')
plt.ylabel('no of customer')
plt.title("Customer Exits or Not ")
plt.hist([tenure_churn_yes, tenure_churn_no], color = ['green', 'red'], label = ['churn_yes', 'churn_no'])
plt.legend()

we are replacing the String data into int data for better model prediction

In [ ]:
bank['Gender'].replace(to_replace={'Male':1,'Female':0},inplace=True)
bank['Geography'].replace(to_replace={'France':2,'Spain':1,'Germany':0},inplace=True)
bank.head()

printing all datatype columns

In [ ]:
bank.dtypes

scaleing our data

In [ ]:
#  to scaleing our model using min_max scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scale_col = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
bank[scale_col] = scaler.fit_transform(bank[scale_col])

In [ ]:
# Now, We need to normalize the columns with different ranges into a same range. This improves the accuracy of the our model.

for i in bank.columns:
  print(f'{i}:{bank[i].unique()}')

<!--  -->

# Creating Model and Training
In the following we want to split the data and train a NN model.


In [ ]:
#  dividing column in x and y
#  and spilt data set into train set and training set using train_test_split
x = bank.drop('Exited',axis='columns')
y = bank['Exited']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=5)

# Building ANN Model

In [ ]:
#

from tensorflow import keras
import tensorflow as tf

model = keras.Sequential([
    keras.Input(shape=(10, )),
    keras.layers.Dense(10, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(
              optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=50)

In [ ]:
model.predict(X_test)

# Evaluation of the model

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
yp = model.predict(X_test)
X_test[:10]

In [ ]:
y_pred = []
for i in yp:
  if i > 0.5:
    y_pred.append(1)
  else:
    y_pred.append(0)
list(y_pred[:10])

In [ ]:
y_test[:10]

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
print(classification_report(y_test,y_pred))

Let's see the confusion matrix

In [ ]:
import seaborn as sn
cm = tf.math.confusion_matrix(labels=y_test, predictions=y_pred)
plt.figure(figsize=(10, 7))
plt.title('Cunfusion matrix')
sn.heatmap(cm,annot=True,fmt='d')
plt.xlabel('predication')
plt.ylabel('Truth')
plt.show()